In [438]:
# CS180 (CS280A): Project 1 starter Python code

# these are just some suggested libraries
# instead of scikit-image you could use matplotlib and opencv to read, write, and display images

import numpy as np
import skimage as sk
import skimage.io as skio
from skimage import img_as_ubyte
from skimage import feature

In [439]:
def pyramid_speedup(base, img, n, ec):
    if n == 0:
        return (0, 0)

    shift = 2 * np.array(pyramid_speedup(sk.transform.rescale(base, 0.5), sk.transform.rescale(img, 0.5), n - 1, ec))

    img_shifted = np.roll(img, shift[0], axis=0)
    img_shifted = np.roll(img_shifted, shift[1], axis=1)

    max_ssin = -np.inf
    min_dist = np.inf
    new_shift = (0, 0)

    r = 15 if n == 1 else 2

    base_mean = np.mean(base)
    base_variance = np.var(base)

    if ec:
        L = max(np.max(base), np.max(img_shifted)) ^ min(np.min(base), np.min(img_shifted))
        c1 = (0.01 * L) ** 2
        c2 = (0.03 * L) ** 2

    for _x in range(-r, r + 1):
        for _y in range(-r, r + 1):
            shifted = np.roll(img_shifted, _x, axis=0)
            shifted = np.roll(shifted, _y, axis=1)

            if ec:
                shifted_mean = np.mean(shifted)
                shifted_variance = np.var(shifted)

                covariance = np.mean(np.multiply(np.subtract(base, base_mean), np.subtract(shifted, shifted_mean)))

                ssim = ((2 * base_mean * shifted_mean + c1) * (2 * covariance + c2)) / \
                        ((base_mean ** 2 + shifted_mean ** 2 + c1) * (base_variance + shifted_variance + c2))
                
                if ssim > max_ssin:
                    max_ssin = ssim
                    new_shift = (_x, _y)
            else:
                dist = np.sqrt(np.sum(np.square(base - shifted)))

                if dist < min_dist:
                    min_dist = dist
                    new_shift = (_x, _y)

    return (shift[0] + new_shift[0], shift[1] + new_shift[1])


In [440]:
def align_image(imname, tif=True, ec=True):
    # read in the image
    im = skio.imread("data/" + imname + (".tif" if tif else ".jpg"))

    # convert to double (might want to do this later on to save memory)
    im = sk.img_as_float(im)

    # compute the height of each part (just 1/3 of total)
    height = np.floor(im.shape[0] / 3.0).astype(int)

    # separate color channels
    b = im[:height]
    g = im[height : 2 * height]
    r = im[2 * height : 3 * height]

    if ec:
        r_shift = pyramid_speedup(feature.canny(g), feature.canny(r), 5 if tif else 1, ec)
        b_shift = pyramid_speedup(feature.canny(g), feature.canny(b), 5 if tif else 1, ec)
    else:
        r_shift = pyramid_speedup(g, r, 5 if tif else 1, ec)
        b_shift = pyramid_speedup(g, b, 5 if tif else 1, ec)

    print(imname)
    print(r_shift)
    print(b_shift)

    ar = np.roll(r, r_shift[0], axis=0)
    ar = np.roll(ar, r_shift[1], axis=1)

    ab = np.roll(b, b_shift[0], axis=0)
    ab = np.roll(ab, b_shift[1], axis=1)

    im_out = np.dstack([ar, g, ab])

    # save the image
    fname = ("ec_" if ec else "") + imname + ".jpg"
    im_out_uint8 = img_as_ubyte(im_out)

    skio.imsave(fname, im_out_uint8)

    # display the image
    # skio.imshow(im_out)
    # skio.show()

In [441]:
tifs = ["lake", "flowers", "river", "church", "harvesters", "emir", "harvesters", "icon", "lady", "melons", "onion_church", "sculpture", "self_portrait", "three_generations", "train"]
jpgs = ["cathedral", "monastery", "tobolsk"]

# Worse versions without ec
for tif in tifs:
    align_image(tif, ec=False)

for jpg in jpgs:
    align_image(jpg, False, False)

# Better versions using ec
for tif in tifs:
    align_image(tif)

for jpg in jpgs:
    align_image(jpg, False)

lake
(-13, 1)
(23, -4)
flowers
(82, 9)
(-75, -19)
river
(80, -6)
(20, 6)
church
(33, -5)
(0, 5)
harvesters
(65, -3)
(-118, 3)
emir
(116, 9)
(3, -7)
harvesters
(65, -3)
(-118, 3)
icon
(48, 5)
(-42, -16)
lady
(44, -10)
(-57, 6)
melons
(96, 3)
(-83, -4)
onion_church
(57, 10)
(-52, -22)
sculpture
(107, -16)
(-33, 11)
self_portrait
(98, 7)
(-50, 2)
three_generations
(57, 0)
(-52, -5)
train
(111, 1)
(-111, 7)
cathedral
(7, 0)
(-1, 1)
monastery
(6, 1)
(6, 0)
tobolsk
(4, 1)
(-3, -2)
lake
(-12, 3)
(20, -6)
flowers
(83, 13)
(-76, -19)
river
(3, 167)
(-22, -23)
church
(33, -8)
(-25, -4)
harvesters
(65, -3)
(-64, -6)
emir
(57, 17)
(-49, -23)
harvesters
(65, -3)
(-64, -6)
icon
(48, 5)
(-38, -16)
lady
(64, -9)
(-38, 13)
melons
(96, 4)
(-89, -10)
onion_church
(56, 48)
(-52, -24)
sculpture
(107, -16)
(-33, 11)
self_portrait
(98, 8)
(-77, -28)
three_generations
(59, -6)
(-56, -11)
train
(44, 30)
(-48, -2)
cathedral
(7, 1)
(-5, -2)
monastery
(6, 1)
(3, -2)
tobolsk
(4, 1)
(-3, -2)
